In [22]:
import pandas as pd
import json
import datetime
import requests
import numpy as np
import math

### List of cities and Ticketmaster codes
City_List = 
    'Denver' : 264,
    'San Francisco Bay': 382,
    'Portland': 362,
    'Los Angeles': 27,
    'Las Vegas' : 14,
    'Phoenix': 36, 
    'Seattle': 42,
    'Austin' : 40, 
    'Houston': 22,
    'Dallas' : 5,
    'Chicago': 3,
    'Nashville': 31,
    'Atlanta': 10,
    'Boston' : 11,
    'New York': 35,
    'Washington DC': 47,
    'Miami' : 15
### Ticketmaster API info    
base_url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US&apikey={apikey}'
api_key = 'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'
dmaId = [382,362,264]
marketId =  [42,27,14,36,40,22,5,3,31,10,11,35,47,15]



### Define Functions

Define functions for pulling event data from Ticketmaster API and formatting dataframe

In [23]:
#DMA and Market numbers
dma_list = [382,362,264]
mkt_list =  [42,27,14,36,40,22,5,3,31,10,11,35,47,15]

# Function to get the number of pages of ticketmaster data
def get_number_of_TM_pages(dma=None,market=None,source='ticketmaster,frontgate'):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'dmaId': dma, 
               'marketId': market, 
               'source': source,
               'classificationName': 'music',
               'size': '200',
               'apikey':'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'}
    r = requests.get(url,params=payload,verify=True)
    json_obj = json.loads(r.text)
    return json_obj['page']['totalPages']

#Get TicketMaster data, return a dataframe
def getTicketMasterData(dma=None,market=None,page=None, source='ticketmaster,frontgate'):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'dmaId': dma, 
               'marketId': market, 
               'source': source,
               'classificationName' : 'music',
               'sort': 'date,name,asc',
               'size': '200',
               'page': page,
               'apikey':'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'}
    r = requests.get(url,params=payload,verify=True)
    json_response = json.loads(r.text)
    event_info = []
    for event in json_response.get('_embedded',{}).get('events',{}):
         event_info.append({
            'TM_id': event.get('id',{}),
            'TM_name' : event.get('name',{}),
            'TM_artist': list(attraction.get('name') for attraction in event['_embedded'].get('attractions',{})),
            'TM_venue' : list(venue.get('name') for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_city' : list(venue.get('city',{}).get('name') for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_state' : list(venue.get('state',{}).get('stateCode') for venue in event['_embedded'].get('venues',{}))[0],
            'TM_description' : event.get('description',{}),
            'TM_more_info' : event.get('additionalInfo',{}),
            'TM_start_date' : event.get('dates',{}).get('start',{}).get('dateTime',{}),
            'TM_timezone' : event.get('dates',{}).get('timezone'),
            'TM_span_multiple_days' : event.get('dates',{}).get('spanMultipleDays'),
            'TM_presale_date_start' : list(presale.get('startDateTime',{}) for presale in event.get('sales').get('presales',{})),
            'TM_presale_date_end' : list(presale.get('endDateTime',{}) for presale in event.get('sales').get('presales',{})),
            'TM_sale_date_start' : event.get('sales',{}).get('public',{}).get('startDateTime'),
            'TM_FV_prices': event.get('priceRanges'),
            'TM_promoter': event.get('promoter',{}).get('name'),
            'TM_genre' : event.get('classifications'),
            'TM_place' : event.get('place')
         })
    tmDF = pd.DataFrame(event_info)
    return tmDF

#Convert timedate information from UTC to local time
def convert_times(df,times_list,tz_col):
    #Loop through each timezone
    df_list = []
    for tz in df[tz_col].unique():
        #Filter rows by timezone
        mask = (df[tz_col] == tz)
        df_local = df.loc[mask]
        #Loop through each datetime row
        for col in times_list:
            #Convert each column to datetime series, localize to UTC and then convert to proper timezone
            df_local[col] = pd.to_datetime(df_local[col],errors='coerce').dt.tz_localize('UTC').dt.tz_convert(tz)
            #Convert each column to datetime series, localize to UTC and then convert to proper timezone
        df_list.append(df_local)
    df = pd.concat(df_list, axis=0)
    return df

#Explode out columns with nested information
def explode(df,col,index_col):
    df1 = df[col].apply(pd.Series)
    df1 = df1[0].apply(pd.Series)
    df1.index = df[index_col]
    df = df.merge(df1, how='left', left_on=index_col, right_index=True)
    df = df.drop(col,axis=1)
    return df

## Query Ticketmaster face value data
### Pull the data and create the data frame
For each city/market, get the number of pages of data, pull all data, and then concatanate the dataframes

In [24]:
fv_df_list = []

#Create dictionaries of dma/mkt numbers, and number of pages of data
dma_dict = dict()
mkt_dict = dict()
#Get number of pages for face value dma items
for dma in dma_list:
    dma_dict[dma] = get_number_of_TM_pages(dma=dma)
    
#Get DMA data
for dma in dma_dict.keys():
    for page in range(1,dma_dict[dma]+1):
        fv_df_list.append(getTicketMasterData(dma=dma,page=page))
    
#Get number of pages for face value market items
for mkt in mkt_list:
    mkt_dict[mkt] = get_number_of_TM_pages(market=mkt)
    
#Get market data
for mkt in mkt_dict.keys():
    for page in range(1,mkt_dict[mkt]+1):
        fv_df_list.append(getTicketMasterData(market=mkt,page=page))
        
#Combine all dataframes for full data on events and face_value prices
fv_df = pd.concat(fv_df_list,axis=0)

fv_df.head()

,TM_FV_prices,TM_artist,TM_description,TM_genre,TM_id,TM_more_info,TM_name,TM_place,TM_presale_date_end,TM_presale_date_start,TM_promoter,TM_sale_date_start,TM_span_multiple_days,TM_start_date,TM_timezone,TM_venue,TM_venue_city,TM_venue_state
0,None,[],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1atZAGEC7oZZe2v,{},Gospel Brunch featuring the Jackie Tolbert Gos...,None,[],[],None,2017-09-08T03:50:01Z,False,2017-12-17T20:30:00Z,America/Los_Angeles,Blue Note Napa,Napa,CA
1,"[{'type': 'standard', 'currency': 'USD', 'min'...",[Irish Christmas],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",G5vYZfnJTUdpA,{},An Irish Christmas,None,[2017-05-26T16:00:00Z],[2017-05-19T17:00:00Z],PROMOTED BY VENUE,2017-05-26T17:00:00Z,False,2017-12-17T22:00:00Z,America/Los_Angeles,William Saroyan Theatre Fresno Convention & En...,Fresno,CA
2,None,[Darryl Worley],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1atZAGpAa0oZe82,{},Darryl Worley,None,[],[],None,2017-08-22T19:00:01Z,False,2017-12-18T02:00:00Z,America/Los_Angeles,The Saddle Rack,Fremont,CA
3,"[{'type': 'standard', 'currency': 'USD', 'min'...",[Big Head Todd & the Monsters],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1AvjZfSGkSdEPdh,{},Big Head Todd & the Monsters,None,[],[],PROMOTED BY VENUE,2017-10-06T17:00:00Z,False,2017-12-18T04:00:00Z,America/Los_Angeles,Crystal Bay Club Casino Crown Room,Crystal Bay,NV
4,None,"[The Grouch, DJ Fresh]",{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1Ae0ZfVGkl_T80L,{},How the Grouch Stole Christmas with the Grouch...,None,[],[],None,2017-10-13T17:00:01Z,False,2017-12-18T04:00:00Z,America/Los_Angeles,Strummer's,Fresno,CA


### Convert datetime columns to datetime objects in  proper timezones
Currently the four columns with datetime information are a mess. Some are of type dict, some are nested in lists, and all are in the UTC timezone. We can look and see that in timedate columns with lists of dates, the timedates are identitical, so we can unnest datetimes by simply taking the first element of the list.

The 3 things we need to accomplish:
    1. Convert all datetime columns to dtype string, and unnest 'TM_presale_date_end' and 'TM_presale_date_start' columns
    2. Convert all columns to datetime series localized to UTC (Done in the convert_times function)
    3. Filter rows by timezone, and loop over each datetime column to convert objects to their proper timezone (Done in the convert_times function)

In [25]:
# 1. Convert all datetime columns to dtype string, and unnest objects
fv_df['TM_presale_date_end'] = fv_df['TM_presale_date_end'].str[0]
fv_df['TM_presale_date_start'] = fv_df['TM_presale_date_start'].str[0]
fv_df['TM_start_date'] = fv_df['TM_start_date'].astype(str)
fv_df['TM_sale_date_start'] = fv_df['TM_sale_date_start'].astype(str)


In [26]:
# 2. Convert all columns to datetime series localized to UTC
# 3. Filter rows by timezone and create a new dataframe per timezone, loop over each datetime column to convert objects to their proper timezone 
col_list = ['TM_presale_date_end','TM_presale_date_start','TM_start_date','TM_sale_date_start']
fv_df = convert_times(df=fv_df,times_list=col_list,tz_col='TM_timezone')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Explode nested rows
Unnest information in prices and genre column

In [27]:
fv_df.head()

,TM_FV_prices,TM_artist,TM_description,TM_genre,TM_id,TM_more_info,TM_name,TM_place,TM_presale_date_end,TM_presale_date_start,TM_promoter,TM_sale_date_start,TM_span_multiple_days,TM_start_date,TM_timezone,TM_venue,TM_venue_city,TM_venue_state
0,None,[],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1atZAGEC7oZZe2v,{},Gospel Brunch featuring the Jackie Tolbert Gos...,None,NaT,NaT,None,2017-09-07 20:50:01-07:00,False,2017-12-17 12:30:00-08:00,America/Los_Angeles,Blue Note Napa,Napa,CA
1,"[{'type': 'standard', 'currency': 'USD', 'min'...",[Irish Christmas],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",G5vYZfnJTUdpA,{},An Irish Christmas,None,2017-05-26 09:00:00-07:00,2017-05-19 10:00:00-07:00,PROMOTED BY VENUE,2017-05-26 10:00:00-07:00,False,2017-12-17 14:00:00-08:00,America/Los_Angeles,William Saroyan Theatre Fresno Convention & En...,Fresno,CA
2,None,[Darryl Worley],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1atZAGpAa0oZe82,{},Darryl Worley,None,NaT,NaT,None,2017-08-22 12:00:01-07:00,False,2017-12-17 18:00:00-08:00,America/Los_Angeles,The Saddle Rack,Fremont,CA
3,"[{'type': 'standard', 'currency': 'USD', 'min'...",[Big Head Todd & the Monsters],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1AvjZfSGkSdEPdh,{},Big Head Todd & the Monsters,None,NaT,NaT,PROMOTED BY VENUE,2017-10-06 10:00:00-07:00,False,2017-12-17 20:00:00-08:00,America/Los_Angeles,Crystal Bay Club Casino Crown Room,Crystal Bay,NV
4,None,"[The Grouch, DJ Fresh]",{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1Ae0ZfVGkl_T80L,{},How the Grouch Stole Christmas with the Grouch...,None,NaT,NaT,None,2017-10-13 10:00:01-07:00,False,2017-12-17 20:00:00-08:00,America/Los_Angeles,Strummer's,Fresno,CA


In [28]:
#Explode price column
fv_df = explode(df=fv_df,col='TM_FV_prices',index_col='TM_id')
fv_df.head()

/usr/local/lib/python3.6/site-packages/pandas/core/indexes/api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/usr/local/lib/python3.6/site-packages/pandas/core/indexes/api.py:57: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)


,TM_artist,TM_description,TM_genre,TM_id,TM_more_info,TM_name,TM_place,TM_presale_date_end,TM_presale_date_start,TM_promoter,...,TM_start_date,TM_timezone,TM_venue,TM_venue_city,TM_venue_state,0,currency,max,min,type
0,[],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1atZAGEC7oZZe2v,{},Gospel Brunch featuring the Jackie Tolbert Gos...,None,NaT,NaT,None,...,2017-12-17 12:30:00-08:00,America/Los_Angeles,Blue Note Napa,Napa,CA,NaN,NaN,NaN,NaN,NaN
1,[Irish Christmas],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",G5vYZfnJTUdpA,{},An Irish Christmas,None,2017-05-26 09:00:00-07:00,2017-05-19 10:00:00-07:00,PROMOTED BY VENUE,...,2017-12-17 14:00:00-08:00,America/Los_Angeles,William Saroyan Theatre Fresno Convention & En...,Fresno,CA,NaN,USD,60.00,25.00,standard
2,[Darryl Worley],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1atZAGpAa0oZe82,{},Darryl Worley,None,NaT,NaT,None,...,2017-12-17 18:00:00-08:00,America/Los_Angeles,The Saddle Rack,Fremont,CA,NaN,NaN,NaN,NaN,NaN
3,[Big Head Todd & the Monsters],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1AvjZfSGkSdEPdh,{},Big Head Todd & the Monsters,None,NaT,NaT,PROMOTED BY VENUE,...,2017-12-17 20:00:00-08:00,America/Los_Angeles,Crystal Bay Club Casino Crown Room,Crystal Bay,NV,NaN,USD,25.69,25.69,standard
4,"[The Grouch, DJ Fresh]",{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1Ae0ZfVGkl_T80L,{},How the Grouch Stole Christmas with the Grouch...,None,NaT,NaT,None,...,2017-12-17 20:00:00-08:00,America/Los_Angeles,Strummer's,Fresno,CA,NaN,NaN,NaN,NaN,NaN


In [36]:
fv_df'TM_genre'].keys()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


Int64Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0],
           dtype='int64')

In [64]:
#Unnest genre info and clean up data in exploded columns
#fv_df = explode(fv_df,'TM_genre','TM_id')
# for col in ['genre','subType']:
#     fv_df[col] = fv_df[col].map(lambda x: x.get('name',{}))
# fv_df.head()

## Query TicketMaster resale data and merge with face value data
Next we will query TicketMaster's resale ticket data and then merge it with the face value dataframe

In [65]:
#Query resale data from the Ticketmaster API

rv_df_list = []

#Create dictionaries of dma/mkt numbers, and number of pages of data
dma_rv_dict = dict()
mkt_rv_dict = dict()
#Get number of pages for resale value dma items
for dma in dma_list:
    dma_rv_dict[dma] = get_number_of_TM_pages(dma=dma,source='tmr')
    
#Get DMA data
for dma in dma_rv_dict.keys():
    for page in range(1,dma_dict[dma]+1):
        fv_df_list.append(getTicketMasterData(dma=dma,page=page,source='tmr'))
    
#Get number of pages for resale value market items
for mkt in mkt_list:
    mkt_rv_dict[mkt] = get_number_of_TM_pages(market=mkt,source='tmr')
    
#Get market data
for mkt in mkt_rv_dict.keys():
    for page in range(1,mkt_dict[mkt]+1):
        fv_df_list.append(getTicketMasterData(market=mkt,page=page,source='tmr'))
        
#Combine all dataframes for full data on events and resale value prices
rv_df = pd.concat(fv_df_list,axis=0)
rv_df.head()

,TM_FV_prices,TM_artist,TM_description,TM_genre,TM_id,TM_more_info,TM_name,TM_place,TM_presale_date_end,TM_presale_date_start,TM_promoter,TM_sale_date_start,TM_span_multiple_days,TM_start_date,TM_timezone,TM_venue,TM_venue_city,TM_venue_state
0,None,[],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1atZAGEC7oZZe2v,{},Gospel Brunch featuring the Jackie Tolbert Gos...,None,[],[],None,2017-09-08T03:50:01Z,False,2017-12-17T20:30:00Z,America/Los_Angeles,Blue Note Napa,Napa,CA
1,"[{'type': 'standard', 'currency': 'USD', 'min'...",[Irish Christmas],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",G5vYZfnJTUdpA,{},An Irish Christmas,None,[2017-05-26T16:00:00Z],[2017-05-19T17:00:00Z],PROMOTED BY VENUE,2017-05-26T17:00:00Z,False,2017-12-17T22:00:00Z,America/Los_Angeles,William Saroyan Theatre Fresno Convention & En...,Fresno,CA
2,None,[Darryl Worley],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1atZAGpAa0oZe82,{},Darryl Worley,None,[],[],None,2017-08-22T19:00:01Z,False,2017-12-18T02:00:00Z,America/Los_Angeles,The Saddle Rack,Fremont,CA
3,"[{'type': 'standard', 'currency': 'USD', 'min'...",[Big Head Todd & the Monsters],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1AvjZfSGkSdEPdh,{},Big Head Todd & the Monsters,None,[],[],PROMOTED BY VENUE,2017-10-06T17:00:00Z,False,2017-12-18T04:00:00Z,America/Los_Angeles,Crystal Bay Club Casino Crown Room,Crystal Bay,NV
4,None,"[The Grouch, DJ Fresh]",{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1Ae0ZfVGkl_T80L,{},How the Grouch Stole Christmas with the Grouch...,None,[],[],None,2017-10-13T17:00:01Z,False,2017-12-18T04:00:00Z,America/Los_Angeles,Strummer's,Fresno,CA


### Extract price information and merge it with face value data in a new dataframe

In [66]:
# Explode out resale price information
rv_df = explode(rv_df,'TM_FV_prices','TM_id')
rv_df.rename(columns={'min': 'tmr_min', 'max': 'tmr_max'}, inplace=True)
# Create a new dataframe combining face value and resale value information, joined on ticketmaster event ID
joined_df = fv_df.join(rv_df[['tmr_min','tmr_max']],on='TM_id',how='outer')
joined_df.head()

/usr/local/lib/python3.6/site-packages/pandas/core/indexes/api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/usr/local/lib/python3.6/site-packages/pandas/core/indexes/api.py:57: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)


,TM_artist,TM_description,TM_id,TM_more_info,TM_name,TM_place,TM_presale_date_end,TM_presale_date_start,TM_promoter,TM_sale_date_start,...,min,type_x,genre,primary,segment,subGenre,subType,type_y,tmr_min,tmr_max
0,[],{},1atZAGEC7oZZe2v,{},Gospel Brunch featuring the Jackie Tolbert Gos...,None,NaT,NaT,None,2017-09-07 20:50:01-07:00,...,NaN,NaN,Religious,True,"{'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'}",Gospel,"{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}",NaN,NaN
1,[Irish Christmas],{},G5vYZfnJTUdpA,{},An Irish Christmas,None,2017-05-26 09:00:00-07:00,2017-05-19 10:00:00-07:00,PROMOTED BY VENUE,2017-05-26 10:00:00-07:00,...,25.00,standard,World,True,"{'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'}",World,"{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}",NaN,NaN
2,[Darryl Worley],{},1atZAGpAa0oZe82,{},Darryl Worley,None,NaT,NaT,None,2017-08-22 12:00:01-07:00,...,NaN,NaN,Country,True,"{'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'}",Country,"{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}",NaN,NaN
3,[Big Head Todd & the Monsters],{},1AvjZfSGkSdEPdh,{},Big Head Todd & the Monsters,None,NaT,NaT,PROMOTED BY VENUE,2017-10-06 10:00:00-07:00,...,25.69,standard,Rock,True,"{'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'}",Pop,"{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}",NaN,NaN
4,"[The Grouch, DJ Fresh]",{},1Ae0ZfVGkl_T80L,{},How the Grouch Stole Christmas with the Grouch...,None,NaT,NaT,None,2017-10-13 10:00:01-07:00,...,NaN,NaN,Hip-Hop/Rap,True,"{'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'}",Urban,"{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}",NaN,NaN


## Download Resale Data from SeatGeek API
1. Connect to SeatGeek API and download data
2. Merge data with TicketMaster DataFrame

### Define functions

In [67]:
#Connect to SeatGeek API and determine number of pages of information
def get_SeatGeek_Pages():
    url = 'https://api.seatgeek.com/2/events?format=json'
    payload = {'per_page' : 1000,
               'taxonomies.name':'concert',
               'client_id': 'OTU5MDE5MXwxNTEwMzcxNjgyLjIx',
              }
    r = requests.get(url, params=payload,verify=True)
    json_obj = json.loads(r.text)
    #Return the total number of JSON items divided by the number of page to get page count
    return math.ceil(json_obj['meta']['total']/json_obj['meta']['per_page'])

#Connect to SeatGeek API and download JSON data, format it into pandas dataframe
def get_SeatGeek_data(page=1):
    url = 'https://api.seatgeek.com/2/events?format=json'
    payload = {'per_page' : 1000,
               'page' : page,
               'taxonomies.name':'concert',
               'client_id': 'OTU5MDE5MXwxNTEwMzcxNjgyLjIx',
              }
    r = requests.get(url,params=payload,verify=True)
    json_obj = json.loads(r.text)
    info_list = []
    for event in json_obj['events']:
        info_list.append(
         {'SG_listing_count' : event.get('stats',{}).get('listing_count',{}),
         'SG_average_price' : event.get('stats',{}).get('average_price',{}),
         'SG_lowest_price' : event.get('stats',{}).get('lowest_price',{}),
         'SG_highest_price' : event.get('stats',{}).get('highest_price',{}),
         'SG_title' : event.get('title',{}),
         'SG_datetime_local' : event.get('datetime_local',{}),
         'SG_artists' : [performer.get('name',{}) for performer in event.get('performers',{})],
         'SG_artists_score' : [performer.get('score',{}) for performer in event.get('performers',{})],
         'SG_artists_type' : [performer.get('type',{}) for performer in event.get('performers',{})],
         'SG_venue' : event.get('venue',{}).get('name',{}),
         'SG_venue_city' : event.get('venue',{}).get('city',{}),
         'SG_venue_state' : event.get('venue',{}).get('state',{}),
         'SG_venue_score' : event.get('venue',{}).get('score',{})})
    sgdf = pd.DataFrame(info_list)
    return sgdf

In [68]:
sgdf_list = []
total_pages = get_SeatGeek_Pages()
#Loop through the number pages of data and combine data into single dataframe
for pageNum in range(1,total_pages+1):
    sgdf_list.append(get_SeatGeek_data(page=pageNum))
sgdf = pd.concat(sgdf_list,axis=0)
sgdf.head()

,SG_artists,SG_artists_score,SG_artists_type,SG_average_price,SG_datetime_local,SG_highest_price,SG_listing_count,SG_lowest_price,SG_title,SG_venue,SG_venue_city,SG_venue_score,SG_venue_state
0,"[Psycho Killers, Psycho Killers - A Tribute To...","[0.3300000131, 0.3199999928]","[band, band]",86.379997,2017-11-24T21:00:00,106.0,8.0,40.0,Black Friday featuring Psycho Killers (Talking...,Baltimore Soundstage,Baltimore,0.473855,MD
1,"[Guttermouth, Koffin Kats]","[0.349999994, 0.3799999952]","[band, band]",NaN,2017-11-24T19:00:00,NaN,NaN,NaN,Guttermouth with Koffin Kats,Marquis Theater,Denver,0.439128,CO
2,[My Hated Friend],[0.0],[band],NaN,2017-11-24T21:00:00,NaN,NaN,NaN,My Hated Friend,Ottawa Tavern,Toledo,0.000000,OH
3,[Sammy Kershaw],[0.4099999964],[band],NaN,2017-11-24T20:00:00,NaN,NaN,NaN,Sammy Kershaw (21+),L'Auberge Casino & Hotel Baton Rouge,Baton Rouge,0.514433,LA
4,[Grieves],[0.3600000143],[band],105.000000,2017-11-24T20:00:00,106.0,4.0,103.0,Grieves (18+),First Avenue,Minneapolis,0.532622,MN
